In [1]:
!wget https://data.ncl.ac.uk/ndownloader/articles/24574729/versions/1 -O Language.zip
!unzip Language.zip

--2025-05-06 16:56:40--  https://data.ncl.ac.uk/ndownloader/articles/24574729/versions/1
Resolving data.ncl.ac.uk (data.ncl.ac.uk)... 54.195.113.107, 52.49.76.148, 34.241.90.58
Connecting to data.ncl.ac.uk (data.ncl.ac.uk)|54.195.113.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161843534 (154M) [application/zip]
Saving to: ‘Language.zip’

Language.zip        100%[===================>] 154.35M  1.31MB/s    in 1m 54s  

2025-05-06 16:58:34 (1.35 MB/s) - ‘Language.zip’ saved [161843534/161843534]

Archive:  Language.zip
 extracting: metadata                
 extracting: test_y.npy              
 extracting: test_x.npy              
 extracting: train_y.npy             
 extracting: valid_y.npy             
 extracting: valid_x.npy             
 extracting: train_x.npy             
 extracting: README                  


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision import datasets, transforms

################################
#     RESTART     RUNTIME      #
################################
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=24 * 24, output_size=10):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.fc0(x)

In [22]:
hyperparams = {
    "num_epochs": 128,
    "batch_size": 256,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.25}, # 1.0 -> no edges, 0.0 -> all edges
    "choose_thresholds_del": {"fc0": 0.1}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.05,
    "min_delta_epoch_replace": 24,
    "window_size": 5,
    "lr": 1e-3,
    "delete_after": 4,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 2000, # None -> no limit
    "weight_decay": 1e-4,
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 128, batch_size: 256, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.25}, choose_thresholds_del: {'fc0': 0.1}, threshold: 0.05, min_delta_epoch_replace: 24, window_size: 5, lr: 0.001, delete_after: 4, task_type: classification, fully_connected: False, max_to_replace: 2000, weight_decay: 0.0001"

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

train_x = np.load("train_x.npy")
train_y = np.load("train_y.npy")
valid_x = np.load("valid_x.npy")
valid_y = np.load("valid_y.npy")
test_x = np.load('test_x.npy')
test_y = np.load('test_y.npy')

train_tensor_x = torch.tensor(train_x).float().view(-1, 24 * 24)
train_tensor_y = torch.tensor(train_y).long()

valid_tensor_x = torch.tensor(valid_x).float().view(-1, 24 * 24)
valid_tensor_y = torch.tensor(valid_y).long()

test_tensor_x = torch.tensor(test_x).float().view(-1, 24 * 24)
test_tensor_y = torch.tensor(test_y).long()


train_dataset = TensorDataset(train_tensor_x, train_tensor_y)
valid_dataset = TensorDataset(valid_tensor_x, valid_tensor_y)
test_dataset = TensorDataset(test_tensor_x, test_tensor_y)


train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=hyperparams['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'], shuffle=False)

In [23]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

In [24]:
import wandb

wandb.login()
wandb.finish()
run = wandb.init(
    project="self-expanding-nets-Language",
    name=f"trash",
    config=hyperparams
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


acc amount,▇█████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
len_choose,█▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
n_params over train_time,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇▇▇▆▇▆▇▇▇▇▇▇▇▇▇██▇▇▇▇██▇
params amount,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅████████████
params ratio,▂▁▁▁▂▃▃▃▃▃▃▃▃▄██████████████████████████
params to replace amount,▇███▇▆▅▅▅▅▄▄▄▅▄▄███▄██▆▅▆▄▂▃▂▂▅▁▃▄▅▇▇▇▇█
train loss,█▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▁▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▄▄▄▄▄▅▄▄▄▄▄▄▄▄▇▇▇▇▇▇█▇▇▇
train_time over n_params,▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val accuracy,▁▃▃▄▅▆▆▆▆▆▆▆████████████████▇▇▇▇▇▇▇▇▇▇▇▇


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sparse_model.parameters(), lr=hyperparams['lr'], weight_decay=hyperparams['weight_decay'])
train_sparse_recursive(sparse_model, train_loader, val_loader, val_loader, criterion, optimizer, hyperparams, device)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 164.31it/s]


Epoch 1/128, Train Loss: 1.8383, Val Loss: 1.5002, Val Accuracy: 0.7255


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 187.57it/s]


Epoch 2/128, Train Loss: 1.2614, Val Loss: 1.1441, Val Accuracy: 0.7537


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 167.57it/s]


...

Epoch 24/128, Train Loss: 0.4532, Val Loss: 0.5520, Val Accuracy: 0.8059


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 174.61it/s]


Epoch 25/128, Train Loss: 0.4506, Val Loss: 0.5496, Val Accuracy: 0.8064


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 176.98it/s]


Epoch 26/128, Train Loss: 0.4482, Val Loss: 0.5474, Val Accuracy: 0.8065
Chosen edges: tensor([[  0,   0,   0,  ...,   9,   9,   9],
        [  0,   1,   2,  ..., 570, 571, 575]], device='cuda:0') 1362


In [ ]:
_, accuracy = eval_one_epoch(sparse_model, criterion, test_loader, hyperparams['task_type'], device)
params = get_params_amount(sparse_model)

In [ ]:
accuracy, params

In [ ]:
import pandas as pd

data = {
    'Model': ['Ours', 'ResNet-18', 'AlexNet', 'VGG16', 'ConvNext', 'MNASNet', 'DenseNet', 'ResNeXt', 'PC-DARTS', 'DrNAS', 'Bonsai-Net', 'DARTS', ' Bonsai', 'Random'],
    'Accuracy (%)': [47.85, 92.08, 94.87, 92.06, 38.06, 90.51, 93.52, 91.42, 96.60, 97.06, 97.91, 97.07, 34.17, 5],
    'Parameters': [9_302_946, 11_689_512, 61_100_840, 138_357_544, 88_591_464, 4_383_312, 28_681_000, 25_028_904, None, None, None, None, None, None]
}

table = pd.DataFrame(data)

def format_with_commas(x):
    return "{:,}".format(x)

styled_table = (table.style
               .format({'Accuracy (%)': '{:.2f}',
                       'Parameters': format_with_commas})
               .set_properties(**{'text-align': 'center'})
               .set_table_styles([
                   {'selector': 'th', 'props': [('text-align', 'center')]},
                   {'selector': 'caption', 'props': [('font-size', '1.1em')]}
               ])
               .hide(axis='index'))

styled_table